## Part 1: Data Preperation

For some context around the project please look the read_me file.

In this first notebook we will generate the data that we will use to fine tune the Llama model. In this case, I will grab the top 1500 posts (all-time) from the askculinary subreddit. Before anything we will import the notebooks we need. Additionally, you will need to create a reddit user agent to scrap data. 

In [1]:
import pandas as pd
import numpy as np
import praw

In [2]:
reddit = praw.Reddit(
    client_id='insert-client-id',
    client_secret='insert-client-secret',
    user_agent='insert-agent-name'  
)

subreddit_name = 'AskCulinary'

In [9]:
top_posts = reddit.subreddit(subreddit_name).top(time_filter='all', limit=1200)

We will now grab the posts and the top comments from the posts. In doing so, we will append into the lists which we will be using to input into a json file.

In [10]:
titles_list = []
comments_list = []

for post in top_posts:
    #print(f"Title: {post.title}")
    #print(f"URL: {post.url}")
    titles_list.append(post.title)
    post.comments.replace_more(limit=0)
    if post.comments:
        # Sort comments by score and get the top comment
        top_comment = sorted(post.comments, key=lambda comment: comment.score, reverse=True)[0]
        #print(f"Top Comment: {top_comment.body}\n")
        comments_list.append(top_comment.body)
    else:
        comments_list.append('No comments found')
        #print("No comments found\n")
        


In [12]:
df = pd.DataFrame(data = {'post' : titles_list, 'comments' : comments_list})

df.to_csv('data.csv', index = False)

No I put my data into a csv for easier data modification. Our next step will be structure our posts comments pairs to follow what our llama model expects. In this case we will insert the [INST] tokens which will indicate were the questions are being placed

Finally, we will insert the data into the json file. 

In [31]:
df['text'] = df.apply(lambda row: '<s> [INST] ' + row['post'] + ' [/INST] ' + row['comments'], axis=1)

json_lines = df['text'].apply(lambda x: {'text': x}).to_json(orient='records', lines=True, force_ascii=False)

json_lines = json_lines.replace('\\/', '/')

with open('recipe_data.jsonl', 'w', encoding='utf-8') as f:
    f.write(json_lines)


We are now done with the data generation step of the project. If the model was hosted in a production environment, then we can run this script in a weekly basis were we can input more recent posts from the week. In this way the model will continue to build knowledge. For this proof of concept however, I will just use this initial data for the pretraining step. 